In [1]:
import io
import os
import torch
import warnings
from tqdm.notebook import tqdm
from torch.utils.data import Dataset, DataLoader
from ml_things import plot_dict, plot_confusion_matrix, fix_text
from sklearn.metrics import classification_report, accuracy_score
from transformers import (set_seed,
                          TrainingArguments,
                          Trainer,
                          GPT2Config,
                          GPT2Tokenizer,
                          AdamW, 
                          get_linear_schedule_with_warmup,
                          GPT2ForSequenceClassification)

warnings.filterwarnings('ignore')

C:\Users\dark_knight\anaconda3\envs\emotion\lib\site-packages\matplotlib\__init__.py:200: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\dark_knight\anaconda3\envs\emotion\lib\site-packages\setuptools\_distutils\version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\dark_knight\anaconda3\envs\emotion\lib\site-packages\matplotlib\__init__.py:200: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  if LooseVersion(module.__version__) < minver:
C:\Users\dark_knight\anaconda3\envs\emotion\lib\site-packages\setuptools\_distutils\version.py:346: DeprecationWarning: distutils Version classes are deprecated. Use packaging.version instead.
  other = LooseVersion(other)
C:\Users\dark_knight\anaconda3\envs\emotion\lib\site-packages\matplotlib\__init__.py:200: De

In [2]:
import pandas as pd

In [3]:
import os

dir_path = "C:\\Users\\Mr.wi\\Desktop\mmm\\Machine-Learning\\dataset"
data_list = []
for (root, directories, files) in os.walk(dir_path):
    for file in files:
        if '.csv' in file:
            file_path = os.path.join(root, file)
            data_list.append(pd.read_csv(file_path,encoding="cp949"))
        if '.xlsx' in file:
            file_path = os.path.join(root, file)
            data_list.append(pd.read_excel(file_path))

In [4]:
data_list[0]

IndexError: list index out of range

In [51]:
data = data_list[0].iloc[:,1:3]

for _ in range(1,3):
    data = pd.concat([data,data_list[_].iloc[:,1:3]])
buf = data_list[3].iloc[:,1:]
buf.columns = ['발화문',"상황"]
data = pd.concat([data,buf])

In [52]:
data

,발화문,상황
0,"어, 청소 니가 대신 해 줘!",anger
1,둘 다 청소 하기 싫어. 귀찮아.,anger
2,둘 다 하기 싫어서 화내.,anger
3,그럼 방세는 어떡해.,anger
4,권태긴줄 알았는데 다른 사람이 생겼나보더라고.,sad
...,...,...
55623,얘긴 다 끝났냐? 원예부,중립
55624,"예. 그거 때문에, 부탁이 있......는......데요.",중립
55625,여자 숨겨달라는거면 사절이다.,중립
55626,아무래도 안되나요?,중립
